In [ ]:
catkin_make --only-pkg-with-deps <target_package>

catkin_make --pkg turtlebot


# Unit 2: Topics


What will you learn with this unit?

1. What is a **Subscriber**
> **A subscriber is a node that reads information from a topic.**

2. how to **create one**.
3. How to **create your own message**. 

## **Example 2.2**



In [ ]:
# 1. simple_topic_subscriber.py

#! /usr/bin/env python

import rospy
from std_msgs.msg import Int32 

def callback(msg): 
  print msg.data

rospy.init_node('topic_subscriber')
sub = rospy.Subscriber('counter', Int32, callback)

# spin() simply keeps python from exiting until this node is stopped
rospy.spin()

In [ ]:
# 2. Subscriber call

rostopic echo /counter

In [ ]:
# user ~ $ rostopic echo /counter
WARNING: no messages received and simulated time is active.
Is /clock being published?

And what does this mean? This means that **nobody is publishing into the /counter topic**, so there's no information to be read.

In [ ]:
# 3. 
rostopic pub <topic_name> <message_type> <value>

In [ ]:
# 4. Publishing now topic 

rostopic pub /counter std_msgs/Int32 5

In [ ]:
# rosmsg show std_msgs/Int32

int32 data

In [ ]:
# user ~ $ rostopic echo /counter
WARNING: no messages received and simulated time is active.
Is /clock being published?
data:
5
---

## Explain the code 

everything with more detail, let's explain the code you executed.

In [ ]:
#! /usr/bin/env python

import rospy                                          
from std_msgs.msg import Int32 

def callback(msg):                        # 1. Define a function called 'callback' that receives a parameter 
                                          # named 'msg'
  
    print msg.data                            # Print the value 'data' inside the 'msg' parameter


rospy.init_node('topic_subscriber')          # 2. Initiate a Node called 'topic_subscriber'

                                              # 3. Instance for subscriber
sub = rospy.Subscriber('/counter', Int32, callback)   
              # 1. Create a Subscriber object that will listen to the /counter
              # 2. topic
              # 3. something from the topic class
              # 4. will cal the 'callback' function each time it reads
              

               # Create a loop that will keep the program in execution
rospy.spin()                                       

## **Exercise 2.2**

> * **The odometry of the robot is published by the robot into the /odom topic**.
> * **You will need to figure out what message uses the /odom topic, and how the structure of this message is**.

In [ ]:
$ rostopic info /odom
Type: nav_msgs/Odometry

Publishers:
 * /gazebo (http://rosdscomputer:46617/)

Subscribers: None

In [ ]:
rosmsg show nav_msgs/Odometry

std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string child_frame_id
geometry_msgs/PoseWithCovariance pose
  geometry_msgs/Pose pose
    geometry_msgs/Point position
      float64 x
      float64 y
      float64 z
    geometry_msgs/Quaternion orientation
      float64 x
      float64 y
      float64 z
      float64 w
  float64[36] covariance
geometry_msgs/TwistWithCovariance twist
  geometry_msgs/Twist twist
    geometry_msgs/Vector3 linear
      float64 x
      float64 y
      float64 z
    geometry_msgs/Vector3 angular
      float64 x
      float64 y
      float64 z
  float64[36] covariance

In [ ]:
#! /usr/bin/env python

import rospy
from nav_msgs.msg import Odometry

def callback(msg): 
  print msg          #This will print the whole Odometry message
                     # print msg.header 
                     #This will print the header section of the Odometry message
                    # print msg.pose 
                    # #This will print the pose section of the Odometry message
    
rospy.init_node('odom_sub_node')
sub = rospy.Subscriber('/odom', Odometry, callback)
rospy.spin()                                   

In [ ]:
# This will print the header section of the Odometry message
#! /usr/bin/env python

import rospy
from nav_msgs.msg import Odometry

def callback(msg): 

  print msg.header       
  # print msg.pose    


rospy.init_node('odom_sub_node')
sub = rospy.Subscriber('/odom', Odometry, callback)
rospy.spin()  

## **Exercise 2.3**


## How to Prepare CMakeLists.txt and package.xml for Custom Topic Message Compilation

In order to create a new message, you will need to do the following steps:

1. **Create a directory** named 'msg' inside your package
2. Inside this directory, **create a file named Name_of_your_message.msg** (more information down)
3. **Modify CMakeLists.txt file** (more information down)
4. **Modify package.xml file** (more information down)
5. **Compile**
6. **Use in code**



Add to {Exercice 2.2}, a Python file that creates a publisher that indicates the age of the robot.

> * For that, you'll need to create a new message called **Age.msg**.
> *To see how you can do that, have a look at the detailed description How to prepare CMakeLists.txt and package.xml for custom topic message compilation.

### 1) Create a directory msg in your package.

In [ ]:
roscd <package_name>
mkdir msg

### 2) The Age.msg file must contain this:



In [ ]:
float32 years
float32 months
float32 days

### 3) In CMakeLists.txt

You will have to edit four functions inside CMakeLists.txt:

1. find_package()
2. add_message_files()
3. generate_messages()
4. catkin_package()

In [ ]:
find_package(catkin REQUIRED COMPONENTS
       rospy
       std_msgs
       message_generation   # Add message_generation here, after the other packages
)

In [ ]:
add_message_files(
      FILES
      Age.msg
    ) # Dont Forget to UNCOMENT the parenthesis and add_message_files TOO

**III. generate_messages()**

Here is where the packages needed for the messages compilation are imported.

In [ ]:
generate_messages(
      DEPENDENCIES
      std_msgs
) # Dont Forget to uncoment here TOO

In [ ]:
catkin_package(
      CATKIN_DEPENDS rospy message_runtime   # This will NOT be the only thing here
)

**Note: Keep in mind that the name of the package in the following example is topic_ex, so in your case, the name of the package may be different.**

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(topic_ex)


find_package(catkin REQUIRED COMPONENTS
  std_msgs
  message_generation
)

add_message_files(
  FILES
  Age.msg
)

generate_messages(
  DEPENDENCIES
  std_msgs
)


catkin_package(
  CATKIN_DEPENDS rospy message_runtime
)

include_directories(
  ${catkin_INCLUDE_DIRS}
)

### 4) Modify package.xml

In [ ]:
<build_depend>message_generation</build_depend> 

<build_export_depend>message_runtime</build_export_depend>
<exec_depend>message_runtime</exec_depend>

In [ ]:
<buildtool_depend>catkin</buildtool_depend>
  <build_depend>rospy</build_depend>
  <build_export_depend>rospy</build_export_depend>
  <exec_depend>rospy</exec_depend>

  <build_depend>message_generation</build_depend> 
  <build_export_depend>message_runtime</build_export_depend>
  <exec_depend>message_runtime</exec_depend>

  <build_depend>std_msgs</build_depend>
  <exec_depend>std_msgs</exec_depend>
  <build_export_depend>std_msgs</build_export_depend>

### **OutPUT**

In [ ]:
rosmsg show Age

In [ ]:
[my_package/Age]:
float32 years
float32 months
float32 days

In [ ]:
#! /usr/bin/env python

import rospy
from exercise_23.msg import Age #Import Age message from the exercise_23 package

rospy.init_node('publish_age_node')
pub = rospy.Publisher('/age_info', Age, queue_size=1) #Create a Publisher that will publish in the /age_info topic
rate = rospy.Rate(2)

age = Age() #Create an Age message object
age.years = 5 #Fill the values of the message
age.months = 10 #Fill the values of the message
age.days = 21 #Fill the values of the message

while not rospy.is_shutdown(): 
  pub.publish(age) #Publish the message into the defined topic /age_info
  rate.sleep()

In [ ]:
<launch>
    <node pkg="exercise_23" type="publish_age.py" name="publish_age_node" output="screen" />
</launch>

In [ ]:
rostopic echo /age_info


> 
> 
> 
> 
> 
> 
> 
> 

ROS Topics: http://wiki.ros.org/Topics

ROS Messages: http://wiki.ros.org/Messages

msg Files: http://wiki.ros.org/msg

Publisher and Subscriber 1: http://wiki.ros.org/ROS/Tutorials/WritingPublisherSubscriber%28python%29

Publisher and Subscriber 2: http://wiki.ros.org/ROS/Tutorials/ExaminingPublisherSubscriber

## Exercise 2.4 

Smart robot avoid obstacles 


In [ ]:
# laser_read.py 

#! /usr/bin/env python

import rospy                                          
from sensor_msgs.msg import LaserScan

def callback(msg):                        # 1. Define a function called 'callback' that receives a parameter 
                                          
 
    print msg.ranges[360]                       


rospy.init_node('laser_subscriber')          # 2. Initiate a Node called 'topic_subscriber'

                                          
sub = rospy.Subscriber('/kobuki/laser/scan', LaserScan, callback)   
             
                  
rospy.spin()

In [ ]:
# robot_move_forward.py 
#! /usr/bin/env python

import rospy
from geometry_msgs.msg import Twist

rospy.init_node('move_robot_forward')
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

rate = rospy.Rate(2)
move = Twist()
move.angular.z = 0.5 #Move the with an angular velocity in the z axis

while not rospy.is_shutdown(): 
  pub.publish(move)
  rate.sleep()


In [ ]:
# robot_left.py 
#! /usr/bin/env python

import rospy
from geometry_msgs.msg import Twist

rospy.init_node('move_robot_left')
pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)


rate = rospy.Rate(2)
move = Twist()
move.linear.x = 0.5 #Move the robot with a linear velocity in the x axis
move.angular.z = 0.5 #Move the with an angular velocity in the z axis

while not rospy.is_shutdown(): 
  pub.publish(move)
  rate.sleep()